## 1\Setup

In [ ]:
# %pip install openai
# %pip install pandas
# %pip install scikit-learn
# %pip install matplotlib
# %pip install kagglehub

In [ ]:
import os
import ast
import pandas as pd
import numpy as np
import kagglehub
import random
from copy import deepcopy

from openai import OpenAI
import os
import re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import json
import matplotlib

In [ ]:
datagen_model = "gpt-4o"
api_key = 'xxx' 

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", api_key))

ClusterSize = 5 # For each gpt request, access BatchSize misconceptions

## 2\MisconceptionName Mapping

In [ ]:
# data_dir = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
data_dir = "D:\VSCode\Kaggle_eedi_sorted\data"
df = pd.read_csv(os.path.join(data_dir, "train.csv"))
content_df = pd.read_csv(os.path.join(data_dir, "misconception_mapping.csv"))

content_df["MisconceptionId"] = content_df["MisconceptionId"].astype(str)
id2name = dict(zip(content_df["MisconceptionId"], content_df["MisconceptionName"]))

In [ ]:
# i= 0
# print('ConstructName:',df.loc[i,'ConstructName'])
# print('SubjectName:',df.loc[i,'SubjectName'])
# print('QuestionText:',df.loc[i,'QuestionText'])
# print('AnswerAText:',df.loc[i,'AnswerAText'])
# print('AnswerBText:',df.loc[i,'AnswerBText'])
# print('AnswerCText:',df.loc[i,'AnswerCText'])
# print('AnswerDText:',df.loc[i,'AnswerDText'])
# print('CorrectAnswer:',df.loc[i,'CorrectAnswer'])
# print('MisconceptionAName:',df.loc[i,'MisconceptionAName'])
# print('MisconceptionBName:',df.loc[i,'MisconceptionBName'])
# print('MisconceptionCName:',df.loc[i,'MisconceptionCName'])
# print('MisconceptionDName:',df.loc[i,'MisconceptionDName'])

In [ ]:
misconception_ids = set()
for col in ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"]:
    misconception_ids.update(df[col].dropna().astype(int).unique())
misconception_ids = list(map(str, misconception_ids))
print(f"Number of unique MisconceptionIds: {len(misconception_ids)}")

In [ ]:
missing_ids = content_df[~content_df["MisconceptionId"].isin(misconception_ids)]["MisconceptionId"].unique().tolist()
len(missing_ids)

In [ ]:
all_ids = missing_ids + misconception_ids
len(all_ids)

In [ ]:
misconception_ids = set()
for col in ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"]:
    misconception_ids.update(df[col].dropna().astype(int).unique())
misconception_ids = list(map(str, misconception_ids))
print(f"Number of unique MisconceptionIds: {len(misconception_ids)}")

In [ ]:
df = df.join(content_df,on='MisconceptionAId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionAName'
df = df.join(content_df,on='MisconceptionBId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionBName'
df = df.join(content_df,on='MisconceptionCId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionCName'
df = df.join(content_df,on='MisconceptionDId')
df = df.drop('MisconceptionId',axis=1)
df.columns.values[-1] = 'MisconceptionDName'

In [ ]:
content_df[~content_df["MisconceptionId"].isin(misconception_ids)]

## 3\ Get similar present misconceptions

In [ ]:
# !pip install sentence-transformers

- find most similar misconceptions for missing misconceptions

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# 加载 BGE 模型
model = SentenceTransformer("BAAI/bge-large-en-v1.5")

# 假设有两个 Misconception 的文本描述
misconception_text_1 = "Assumes a sequence is linear"
misconception_text_2 = "Thinks terms in linear sequence are in direct proportion"

# 生成 embedding
emb1 = model.encode(misconception_text_1, normalize_embeddings=True)
emb2 = model.encode(misconception_text_2, normalize_embeddings=True)

# 计算余弦相似度
similarity = cosine_similarity([emb1], [emb2])[0][0]
print(f"Similarity: {similarity:.4f}")


In [ ]:
# 1. 获取所有文本
missing_texts = content_df[content_df['MisconceptionId'].isin(missing_ids)]['MisconceptionName'].tolist()
present_texts = content_df[content_df['MisconceptionId'].isin(misconception_ids)]['MisconceptionName'].tolist()

# 2. 批量生成 embedding
missing_embeddings = model.encode(missing_texts, normalize_embeddings=True, batch_size=32)
present_embeddings = model.encode(present_texts, normalize_embeddings=True, batch_size=32)

# 3. 计算相似度矩阵
similarity_matrix = cosine_similarity(missing_embeddings, present_embeddings)

# 4. 获取TOP K相似的misconceptions
top_indices_matrix = np.argsort(-similarity_matrix, axis=1)[:, :ClusterSize-1]  # 从大到小

missing_similar_misconceptions = {}
for i, missing_id in enumerate(missing_ids):
    top_misconception_ids = [misconception_ids[j] for j in top_indices_matrix[i]]
    missing_similar_misconceptions[missing_id] = top_misconception_ids



In [ ]:
i = 900
print("Missing Misconception:",content_df.loc[content_df['MisconceptionId']==missing_ids[i],'MisconceptionName'].item())
for j in range(ClusterSize-1):
    print("Similar Misconceptions:",content_df.loc[content_df['MisconceptionId']==missing_similar_misconceptions[missing_ids[i]][j],'MisconceptionName'].item())

## 4\prompt and data

In [ ]:
def get_mcq_json(mid,df):
    # 找到包含某misconception的MCQ
    for i in range(len(df)):
        filtered_df = df.loc[(
    (df['MisconceptionAId'] == float(mid)) |(df['MisconceptionBId'] == float(mid)) |(df['MisconceptionCId'] == float(mid)) |(df['MisconceptionDId'] == float(mid))
    ),['SubjectName','ConstructName','QuestionText','AnswerAText','MisconceptionAName','AnswerBText','MisconceptionBName',\
        'AnswerCText','MisconceptionCName','AnswerDText','MisconceptionDName']]
    # 从 filtered_df 中随机抽取一行
    mcq_json = filtered_df.sample(n=1, random_state=42).iloc[0].to_json(orient='index')  # random_state 可保证可复现
    return mcq_json

In [ ]:
def get_res(mids,df):
  # mids，第一条是missing_misconception_id，其他是present最相关的misconception_id
    misconception_rows = []
    mcq_rows = []
    missing_misconception = content_df.loc[content_df['MisconceptionId']==mids[0],'MisconceptionName'].item()
    for m in mids:
        misconception_rows.append(str(mids.index(m)+1) + '. ' +content_df.loc[content_df['MisconceptionId']==m,'MisconceptionName'].item())
        if mids.index(m)>0:
          mcq_rows.append(get_mcq_json(m,df)) #以字符串形式输出json
    cluster_misconceptions = '\n'.join(misconception_rows)
    print(cluster_misconceptions)
    reference_mcqs = '\n'.join(mcq_rows) #input
    print(reference_mcqs)
    num_mcqs = 2 # 每个missing_misconception 生成num_mcqs个MCQs
    
    prompt = f"""Your task is to generate Multiple Choice Questions (MCQs) that diagnose the following misconceptions:
<misconceptions>
{cluster_misconceptions}
</misconceptions>

Here are reference MCQs that demonstrate the questions and misconceptions:
<reference_mcqs>
{reference_mcqs}
</reference_mcqs>

First, in your first key-value pair (key is "reference_analysis"), analyze the reference MCQs carefully:
1. Understand how to derive the correct answer and how the wrong answers map to respective misconceptions
2. Note the style, difficulty level, and precision of language used

Then, in your first key-value pair (key is "MCQ_generation"), generate {num_mcqs} new MCQs that diagnose this conception "{missing_misconception}" , following these guidelines:
1. brainstorm mathematical contexts where this misconception commonly appears
2. generate the problem, solve it yourself, design 1 right answer, and 3 wrong answers leading to specific misconceptions
3. Make questions challenging enough that students must demonstrate real understanding
4. ensure the answers are plausible and stem from genuine misconceptions, not careless errors
5. Use precise mathematical language matching the style of reference MCQs
6. Keep the construct name and subject name as short as possible
7. Follow the json format of reference MCQs.


Additionaly, for each answer of a MCQ, add a key-value pair and write down your reasoning process.
For example: {{'SubjectName':'write subject of this question','ConstructName':'write construct of this question','QuestionText':'write question text',\
'MisconceptionAName':'write down a misconception related to this question as the misconceptionA ','ReasoningA':'write down how misconceptionA leads to answerA','AnswerAText':'write down the answer misonceptionA leads to',\
'MisconceptionBName':'write down a misconception related to this question as the misconceptionB ','ReasoningB':'write down how misconceptionB leads to answerB','AnswerBText':'write down the answer misonceptionB leads to',\
'MisconceptionCName':'write down a misconception related to this question as the misconceptionC ','ReasoningC':'write down how misconceptionC leads to answerC','AnswerCText':'write down the answer misonceptionC leads to',\
'MisconceptionDName':'write down a misconception related to this question as the misconceptionD ','ReasoningD':'write down how misconceptionD leads to answerD','AnswerDText':'write down the answer misonceptionD leads to',\
'CorrectAnswer':'write the correct answer(A/B/C/D)','CorrectAnswerReasoning':'write down why the correct answer is correct'}}
    """
    print(prompt)
    response = client.chat.completions.create(
    model=datagen_model,
    temperature=0.3,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to generate synthetic data."},
        {"role": "user", "content": prompt}
      ]
    )
    res = response.choices[0].message.content
    return res

# V2：
# V3：去掉3. Include other misconceptions mentioned in the <misconceptions> tag in wrong choices if related这个要求
# V4：8. If possible, the news generated {num_mcqs} MCQs should be different from each other.

In [ ]:
# def get_res_o1(mids,df):
#   # mids，第一条是missing_misconception_id，其他是present最相关的misconception_id
#     misconception_rows = []
#     mcq_rows = []
#     missing_misconception = content_df.loc[content_df['MisconceptionId']==mids[0],'MisconceptionName'].item()
#     for m in mids:
#         misconception_rows.append(str(mids.index(m)+1) + '. ' +content_df.loc[content_df['MisconceptionId']==m,'MisconceptionName'].item())
#         if mids.index(m)>0:
#           mcq_rows.append(get_mcq_json(m,df)) #以字符串形式输出json
#     cluster_misconceptions = '\n'.join(misconception_rows)
#     print(cluster_misconceptions)
#     reference_mcqs = '\n'.join(mcq_rows) #input
#     print(reference_mcqs)
#     num_mcqs = 2 # 每个missing_misconception 生成num_mcqs个MCQs
    
#     prompt = f"""Your task is to generate Multiple Choice Questions (MCQs) that diagnose the following misconceptions:
# <misconceptions>
# {cluster_misconceptions}
# </misconceptions>

# Here are reference MCQs that demonstrate the questions and misconceptions:
# <reference_mcqs>
# {reference_mcqs}
# </reference_mcqs>

# First, in your first key-value pair (key is "reference_analysis"), analyze the reference MCQs carefully:
# 1. Understand how to derive the correct answer and how the wrong answers map to respective misconceptions
# 2. Note the style, difficulty level, and precision of language used

# Then, in your first key-value pair (key is "MCQ_generation"), generate {num_mcqs} new MCQs that diagnose this conception "{missing_misconception}" , following these guidelines:
# 1. brainstorm mathematical contexts where this misconception commonly appears
# 2. generate the problem, solve it yourself, design 1 right answer, and 3 wrong answers leading to specific misconceptions
# 3. Make questions challenging enough that students must demonstrate real understanding
# 4. ensure the answers are plausible and stem from genuine misconceptions, not careless errors
# 5. Use precise mathematical language matching the style of reference MCQs
# 6. Keep the construct name and subject name as short as possible
# 7. Follow the json format of reference MCQs.


# Additionaly, for each answer of a MCQ, add a key-value pair and write down your reasoning process.
# For example: {{'SubjectName':'write subject of this question','ConstructName':'write construct of this question','QuestionText':'write question text',\
# 'MisconceptionAName':'write down a misconception related to this question as the misconceptionA ','ReasoningA':'write down how misconceptionA leads to answerA','AnswerAText':'write down the answer misonceptionA leads to',\
# 'MisconceptionBName':'write down a misconception related to this question as the misconceptionB ','ReasoningB':'write down how misconceptionB leads to answerB','AnswerBText':'write down the answer misonceptionB leads to',\
# 'MisconceptionCName':'write down a misconception related to this question as the misconceptionC ','ReasoningC':'write down how misconceptionC leads to answerC','AnswerCText':'write down the answer misonceptionC leads to',\
# 'MisconceptionDName':'write down a misconception related to this question as the misconceptionD ','ReasoningD':'write down how misconceptionD leads to answerD','AnswerDText':'write down the answer misonceptionD leads to',\
# 'CorrectAnswer':'write the correct answer(A/B/C/D)','CorrectAnswerReasoning':'write down why the correct answer is correct'}}
#     """
#     response = client.responses.create(
#     model="o1",
#     reasoning={"effort": "medium"},
#     input=[
#         {"role": "system", "content": "You are a precise data generation assistant for diagnostic MCQs."},
#         {"role": "user", "content": prompt}
#     ]
#     )

#     res = response.choices[0].message.content
#     return res

# # V2：
# # V3：去掉3. Include other misconceptions mentioned in the <misconceptions> tag in wrong choices if related这个要求
# # V4：8. If possible, the news generated {num_mcqs} MCQs should be different from each other.

## 5\ Evaluation

In [ ]:
import pandas as pd

with open('misconceptions_clustered.csv', 'r', encoding='utf-8', errors='ignore') as f:
    missing_misconception_clustered = pd.read_csv(f)


In [ ]:
missing_misconception_clustered.sort_values(by='ClusterId',inplace=True)

In [ ]:
evaluation_misconceptions_df = (
    missing_misconception_clustered
    .groupby('ClusterId', group_keys=False)
    .apply(lambda x: x.sample(n=2, random_state=42))  # random_state 保证可复现
    .reset_index(drop=True)
)


In [ ]:
evaluation_misconceptions_df.head()

In [ ]:
res_list = []
errer_res_list = []
syn_df_list = []
misconception_ids = [int(x) for x in misconception_ids]
missing_ids = [int(x) for x in missing_ids]
syn_df = pd.DataFrame({'SubjectName':[],'ConstructName':[],'QuestionText':[],\
    'AnswerAText':[],'MisconceptionAName':[],'ReasoningA':[],\
    'AnswerBText':[],'MisconceptionBName':[],'ReasoningB':[],\
    'AnswerCText':[],'MisconceptionCName':[],'ReasoningC':[],\
    'AnswerDText':[],'MisconceptionDName':[],'ReasoningD':[],\
    'CorrectAnswer':[],'CorrectAnswerReasoning':[]})

for i in range(len(evaluation_misconceptions_df)):
    mids = str(int(evaluation_misconceptions_df.loc[i,'MisconceptionId'].item()))
    rids = missing_similar_misconceptions[mids]
    res = json.loads(get_res([mids]+rids,df))
    res_list.append(res) # 保存原始的输出结果
    reference_analysis, mcq_data = res['reference_analysis'], res['MCQ_generation']
    try:
        for mcq in mcq_data:
            row = {
                'SubjectName': mcq['SubjectName'],
                'ConstructName': mcq['ConstructName'],
                'QuestionText': mcq['QuestionText'],
                'AnswerAText': mcq['AnswerAText'],
                'MisconceptionAName': mcq['MisconceptionAName'],
                'ReasoningA': mcq['ReasoningA'],
                'AnswerBText': mcq['AnswerBText'],
                'MisconceptionBName': mcq['MisconceptionBName'],
                'ReasoningB': mcq['ReasoningB'],
                'AnswerCText': mcq['AnswerCText'],
                'MisconceptionCName': mcq['MisconceptionCName'],
                'ReasoningC': mcq['ReasoningC'],
                'AnswerDText': mcq['AnswerDText'],
                'MisconceptionDName': mcq['MisconceptionDName'],
                'ReasoningD': mcq['ReasoningD'],
                'CorrectAnswer': mcq['CorrectAnswer'],
                'CorrectAnswerReasoning': mcq['CorrectAnswerReasoning']
            }
            syn_df_list.append(pd.DataFrame([row]))
    except:
        errer_res_list.append(res)
    # break
syn_dfs = pd.concat(syn_df_list)

In [ ]:
syn_dfs.to_excel('syn_data_4.xlsx',index=False)

In [ ]:
mids = str(int(evaluation_misconceptions_df.loc[8,'MisconceptionId'].item()))
rids = missing_similar_misconceptions[mids]
res = get_res([mids]+rids,df)
reference_analysis, mcq_data = res['reference_analysis'], res['MCQ_generation']

## 6\ Large-Scale Production

In [ ]:
# # debug
# rows = []
# i = 0
# mids = missing_ids[BatchSize*i:BatchSize*(i+1)] if BatchSize*(i+1)<=len(missing_ids) else len(missing_ids)
# rids = misconception_ids[BatchSize*i:BatchSize*(i+1)]
# for i in mids:
#     rows.append('Misconception:' +content_df.loc[int(i),'MisconceptionName'])
# cluster_misconceptions = '\n'.join(rows)
# reference_mcqs = gen_reference_mcq(rids,df) #input
# num_mcqs = len(mids) # generate 2 questions for each misconception

# prompt = f"""You are a Mathematics teacher. Your task is to generate Multiple Choice Questions (MCQs) that diagnose the following misconceptions:
# <misconceptions>
# {cluster_misconceptions}
# </misconceptions>

# Here are reference MCQs that demonstrate the questions and misconceptions:
# <reference_mcqs>
# {reference_mcqs}
# </reference_mcqs>

# First, analyze the reference MCQs carefully:
# 1. Understand how to derive the correct answer and how the wrong answers map to respective misconceptions
# 2. Note the style, difficulty level, and precision of language used
# 3. List misconceptions still need coverage

# Then, genrate {num_mcqs} new MCQs that diagnose misconceptions not already covered by the reference MCQs.
# For each needed misconception, in your <reasoning> tag, show your process following these guidelines:
# 1. brainstorm mathematical contexts where it commonly appears
# 2. solve the probelm yourself and then design wrong answers leading to specific miesconceptions
# 3. Make questions challenging enough that students must demonstrate real understanding
# 4. ensure the answers are plausible and stem from genuine misconceptions, not careless errors
# 5. Use precise mathematical language matching the style of reference MCQs
# 5. Keep the construct name and subject name as short as possible

# """
# response = client.chat.completions.create(
# model=datagen_model,
# messages=[
#     {"role": "system", "content": "You are a helpful assistant designed to generate synthetic data."},
#     {"role": "user", "content": prompt}
#     ]
# )
# res = response.choices[0].message.content

In [ ]:
# print(prompt)

In [ ]:
# print(res)

# 7\ LLM-as-judge

In [ ]:
PROBLEM_DATA = str(syn_data_org.loc[2,:].to_json())
prompt = f'''
Here is the problem with both correct and incorrect answers. The suspected misconception is also provided:
{PROBLEM_DATA}

Analyze the problem and provide your evaluation:
1. in "CorrectSolution", solve the problem independently to verify the correct answer
2. in "MisconceptionAReview"/”MisconceptionBReview”/”MisconceptionCReview”/”MisconceptionDReview”, examine how someone holding the suspected misconception would \
approach the problem, trace the logical path from misconception to incorrect answer and identify any gaps or inconsistencies in this connection
3. in "MisconceptionAScore"/”MisconceptionBScore”/”MisconceptionCScore”/”MisconceptionDScore”, score from 0-10 based on these criteria:
   - 10: Perfect alignment - wrong answer is direct result of misconception
   - 8-9: Strong alignment - clear logical path from misconception to answer
   - 5-7: Moderate alignment - connection exists but has some gaps
   - 1-4: Weak alignment - connection is unclear or requires assumptions
   - 0: No alignment - misconception does not explain wrong answer
   for correct answer which you think is correct, score 10, for correct answer which you think is incorrect, score 0
Important guidelines:
- Focus solely on the logical connection between misconception and wrong answer
- Do not speculate about other possible misconceptions
- Be specific about how the misconception leads to the error
- Flag and deduct scores if any assumptions are required to connect misconception to answer
- Consider whether a student with this misconception would consistently arrive at this wrong answer

Output your evaluation results in the following json format:
{{
    "CorrectSolution": "write down the correct solution process",
    "CorrectAnswer": "write the correct answer(A/B/C/D)",
    "MisconceptionAReview": "write down your evaluation though for MisconceptionA",
    "MisconceptionAScore": "write down the score for MisconceptionA",
    "MisconceptionBReview": "write down your evaluation though for MisconceptionB",
    "MisconceptionBScore": "write down the score for MisconceptionB",
    "MisconceptionCReview": "write down your evaluation though for MisconceptionC",
    "MisconceptionCScore": "write down the score for MisconceptionC",
    "MisconceptionDReview": "write down your evaluation though for MisconceptionD",
    "MisconceptionDScore": "write down the score for MisconceptionD",
}}
'''

In [ ]:
response = client.chat.completions.create(
    model=datagen_model,
    temperature=0.1,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "You will analyze if the correct answer is actually correct and how how well an incorrect answer reflects a suspected misconception in a mathematics problem."},
        {"role": "user", "content": prompt}
      ]
    )
res = response.choices[0].message.content

In [ ]:
print(res)